In [ ]:
!fusermount -u /content/drive

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

First we create a directory in which we can download the Synthea application

In [ ]:
!mkdir -p /content/drive/MyDrive/synthea

Now fetch the latest release of the Synthea .jar file from the Synthea github repository

In [ ]:
!wget -O /content/drive/MyDrive/synthea/synthea.jar https://github.com/synthetichealth/synthea/releases/download/master-branch-latest/synthea-with-dependencies.jar

We also need to fetch the synthea.config and the custom IBS module from the Workshop github repository

In [ ]:
!wget -O /content/drive/MyDrive/synthea/synthea.config https://raw.githubusercontent.com/btwooton/arch_workshop_synthetic_data_ws7/main/synthea.config

In [ ]:
!mkdir /content/drive/MyDrive/synthea/local_modules_dir

In [ ]:
!wget -O /content/drive/MyDrive/synthea/local_modules_dir/irritable_bowel_syndrome__ibs.json https://raw.githubusercontent.com/btwooton/arch_workshop_synthetic_data_ws7/main/local_modules_dir/irritable_bowel_syndrome__ibs.json

In [ ]:
!mkdir /content/drive/MyDrive/synthea/local_modules_dir/ibs

In [ ]:
!wget -O /content/drive/MyDrive/synthea/local_modules_dir/ibs/living_with_ibs.json https://raw.githubusercontent.com/btwooton/arch_workshop_synthetic_data_ws7/main/local_modules_dir/ibs/living_with_ibs.json

In [ ]:
# defining a path prefix from which to run the synthea jar file
synthea_prefix = "/content/drive/MyDrive/synthea/"

# Basic Example: Running the Synthea Synthetic Patient Generation Tool
- Here we pass in `-p 100` to specify that we want to simulate 100 patients
- We pass in a specific config file with `-c synthea.config`
- We specify that we want to simulate patients for the state of washington by giving the name `Washington`

In [ ]:
!java -jar "{synthea_prefix}synthea.jar" -c "{synthea_prefix}synthea.config" -p 100 --exporter.baseDirectory="{synthea_prefix}/output" Washington

# Extended Example: Running the Synthea Tool for Multiple Different States
- We can simulate patients for multiple states by invoking the synthea .jar file multiple times
- We specify distinct output directories using the `--exporter.baseDirectory="<directory_path>"` commandline argument

In [ ]:
!java -jar "{synthea_prefix}synthea.jar" -c "{synthea_prefix}synthea.config" -p 10 --exporter.baseDirectory="{synthea_prefix}/output_wa" Washington > /dev/null 2>&1 && \
echo "Finished generating patients for Washington" && \
java -jar "{synthea_prefix}synthea.jar" -c "{synthea_prefix}synthea.config" -p 10 --exporter.baseDirectory="{synthea_prefix}/output_ca" California > /dev/null 2>&1 && \
echo "Finished generating patients for California" && \
java -jar "{synthea_prefix}synthea.jar" -c "{synthea_prefix}synthea.config" -p 10 --exporter.baseDirectory="{synthea_prefix}/output_hi" Hawaii > /dev/null 2>&1 && \
echo "Finished generating patients for Hawaii" && \
java -jar "{synthea_prefix}synthea.jar" -c "{synthea_prefix}synthea.config" -p 10 --exporter.baseDirectory="{synthea_prefix}/output_tx" Texas > /dev/null 2>&1 && \
echo "Finished generating patients for Texas" && \
java -jar "{synthea_prefix}synthea.jar" -c "{synthea_prefix}synthea.config" -p 10 --exporter.baseDirectory="{synthea_prefix}/output_id" Idaho > /dev/null 2>&1 && \
echo "Finished generating patients for Idaho"

# Loading Custom Modules from a Subfolder
- If we want to extend the Synthea tool with our own custom .json modules, we can load these with the `[-d localModulesDirPath]` commandline argument

In [ ]:
!java -jar "{synthea_prefix}synthea.jar" -c "{synthea_prefix}synthea.config" -p 150 \
--exporter.baseDirectory="{synthea_prefix}/output_custom" -d "{synthea_prefix}/local_modules_dir" Washington

## Testing to make sure we have patients with IBS diagnoses


In [ ]:
import pandas as pd

In [ ]:
conditions = pd.read_csv(f"{synthea_prefix}output_custom/csv/conditions.csv")

In [ ]:
ibs = conditions[conditions['DESCRIPTION'].str.contains('irritable bowel', case=False)]

In [ ]:
ibs['DESCRIPTION'].value_counts()